## **Schapen detecteren**

### Inladen van de libraries

In [ ]:
import pandas as pd
import os
from PIL import Image
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import seaborn as sns
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers

### Inladen van data

In [ ]:
def toon_afbeeldingen_van_dataset(map_pad, dataset_naam, aantal=1):
    bestandsnamen = os.listdir(map_pad)
    aantal_afbeeldingen = len(bestandsnamen)  

    print(f"Aantal afbeeldingen in {dataset_naam} map: {aantal_afbeeldingen}")

    for i in range(min(aantal, aantal_afbeeldingen)):
        afbeelding_pad = os.path.join(map_pad, bestandsnamen[i])
        afbeelding = Image.open(afbeelding_pad)
        afbeelding_shape = afbeelding.size  

        plt.imshow(afbeelding)
        plt.title(f'Plaatje {i+1}: {dataset_naam} - Shape: {afbeelding_shape}')
        plt.show()

In [ ]:
# Train map
train_map_pad = 'train'
toon_afbeeldingen_van_dataset(train_map_pad, 'Train')

# Test map
test_map_pad = 'test'
toon_afbeeldingen_van_dataset(test_map_pad, 'Test')

# Valid map
valid_map_pad = 'valid'
toon_afbeeldingen_van_dataset(valid_map_pad, 'Valid')

In [ ]:
# Annotaties inlezen
train_annotaties_path = 'train_annotations.csv'
train_annotaties_df = pd.read_csv(train_annotaties_path)

merge_df = train_annotaties_df.groupby('filename').size().reset_index(name='aantal_schapen')
merge_df.head(5)

In [ ]:
# Functie om afbeeldingen te laden op basis van bestandspaden
def load_images_from_file_paths(file_paths):
    images = []
    for file_path in file_paths:
        try:
            image = Image.open(file_path)
            images.append(image)
        except Exception as e:
            print(f"Fout bij het openen van afbeelding {file_path}: {e}")
    return images

### DataFrames maken

In [ ]:
# Groepen op bestandsnaam en aantal schapen tellen
aantal_schapen_per_afbeelding = train_annotaties_df.groupby('filename').size().reset_index(name='aantal_schapen')

train_files, test_valid_files = train_test_split(aantal_schapen_per_afbeelding['filename'], test_size=0.2, random_state=42)
test_files, valid_files = train_test_split(test_valid_files, test_size=0.5, random_state=42)

# Dataframes voor de train, test en valid sets
train_df = aantal_schapen_per_afbeelding[aantal_schapen_per_afbeelding['filename'].isin(train_files)]
test_df = pd.DataFrame({'filename': test_files})
valid_df = pd.DataFrame({'filename': valid_files})

train_folder_files = [os.path.join(train_map_pad, filename) for filename in os.listdir(train_map_pad) if filename.lower().endswith(('.jpg', '.jpeg', '.png'))]
test_folder_files = [os.path.join(test_map_pad, filename) for filename in os.listdir(test_map_pad) if filename.lower().endswith(('.jpg', '.jpeg', '.png'))]
valid_folder_files = [os.path.join(valid_map_pad, filename) for filename in os.listdir(valid_map_pad) if filename.lower().endswith(('.jpg', '.jpeg', '.png'))]

# Maak dataframes met de werkelijke bestandsnamen
train_df = pd.DataFrame({'filename': train_folder_files})
test_df = pd.DataFrame({'filename': test_folder_files})
valid_df = pd.DataFrame({'filename': valid_folder_files})

# Laad train-, test- en validatieafbeeldingen
train_images = load_images_from_file_paths(train_df['filename'].values)
test_images = load_images_from_file_paths(test_df['filename'].values)
valid_images = load_images_from_file_paths(valid_df['filename'].values)

# Sluit de geopende afbeeldingen om te voorkomen dat er te veel bestanden open blijven
for image in train_images + test_images + valid_images:
    image.close()

# Print de lengtes van de sets
print("Train set lengte:", len(train_df))
print("Test set lengte:", len(test_df))
print("Valid set lengte:", len(valid_df))

### **Data preprocessing**

In [ ]:
train_df

In [ ]:
train_df['filename'] = train_df['filename'].apply(lambda x: os.path.basename(x))
test_df['filename'] = test_df['filename'].apply(lambda x: os.path.basename(x))
valid_df['filename'] = valid_df['filename'].apply(lambda x: os.path.basename(x))

In [ ]:
merge_df['filename'] = merge_df['filename'].apply(lambda x: os.path.normpath(x))
train_df['filename'] = train_df['filename'].apply(lambda x: os.path.normpath(x))

# Merge de dataframes op 'filename'
train_df = pd.merge(train_df, merge_df, on='filename', how='left')
train_df

In [ ]:
display(train_df.head(5))
display(test_df.head(5))
display(valid_df.head(5))

## EDA Afbeeldingen

In [ ]:
min_aantal_schapen = train_df['aantal_schapen'].min()
max_aantal_schapen = train_df['aantal_schapen'].max()

plt.hist(train_df['aantal_schapen'], bins=range(int(min_aantal_schapen), int(max_aantal_schapen)+1))
plt.title('Histogram van aantal schapen per afbeelding')
plt.xlabel('Aantal schapen')
plt.ylabel('Aantal afbeeldingen')
plt.show()

print(train_df["aantal_schapen"].describe())

print("Modus:", train_df["aantal_schapen"].mode())

In [ ]:
sns.boxplot(x=train_df['aantal_schapen'])
plt.title('Boxplot van aantal schapen per afbeelding')
plt.xlabel('Aantal schapen')
plt.show()

### Model

In [ ]:
schapen_teller_model = keras.Sequential([
    layers.Conv2D(16, (3, 3), activation='relu', input_shape=(600, 600, 3)),
    layers.MaxPooling2D(2, 2),
    layers.Conv2D(32, (3, 3), activation='relu'),
    layers.MaxPooling2D(2, 2),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D(2, 2),
    layers.Flatten(),
    layers.Dense(512, activation='relu'),
    layers.Dense(1, activation='linear')])

Het gebruik van de lineaire activatiefunctie in de laatste  neuron is geschikt voor regressieproblemen, waarbij we een continue uitvoer willen voorspellen in plaats van een classificatie. In het geval van het tellen van schapen op afbeeldingen, willen we een getal voorspellen dat het aantal schapen op de afbeelding vertegenwoordigt, wat een continue variabele is.

In [ ]:
## Maken van x_train en y_train
x_train = train_df
y_train = train_df['aantal_schapen']

In [ ]:
# Naar numpy arrays
x_train = np.array(x_train)
y_train = np.array(y_train)

In [ ]:
# Compile  model
schapen_teller_model.compile(optimizer='adam', loss='mean_squared_error')

**Mean Squared Error:**
 
 MSE kwadrateert het verschil tussen de voorspelde waarden en de werkelijke waarden en neemt het gemiddelde over alle voorbeelden. Door het kwadrateren van het verschil worden grote fouten zwaarder gewogen dan kleine fouten, wat nuttig is omdat we willen dat ons model zich meer bewust is van grotere afwijkingen. Voor het tellen van schapen willen we de afwijking tussen het voorspelde aantal schapen en het werkelijke aantal schapen minimaliseren, dus MSE is een wordt daarom onze keus.

**Adam Optimizer:**

De reden voor Adam is omdat Adam een zeer populaire techniek en aansluit bij ons regressieprobleem


In [ ]:
# Train the model
schapen_teller_model.fit(x_train, y_train, epochs=10, batch_size=32)